In [1]:
import torch
import torchvision

from torchvision import models
import transformers
from transformers import PvtModel, AutoImageProcessor

from fir.datasets import deep_fashion_ctsrbm

import os
import pathlib

import numpy as np

from datasets import load_dataset

from tqdm.notebook import tqdm

---

In [2]:
model = PvtModel.from_pretrained("Zetatech/pvt-small-224")

In [3]:
ctsrbm_image_transform = AutoImageProcessor.from_pretrained("Zetatech/pvt-small-224")
ctsrbm_image_transform.size["height"] = 448
ctsrbm_image_transform.size["width"] = 448
ctsrbm_image_transform_corr = lambda t: torch.from_numpy(ctsrbm_image_transform(t).pixel_values[0])
ctsrbm_dataset_dir = os.path.join(pathlib.Path.home(), "data", "DeepFashion", "Consumer-to-shop Clothes Retrieval Benchmark")
ctsrbm_dataset = deep_fashion_ctsrbm.ConsToShopClothRetrBmkImageLoader(ctsrbm_dataset_dir, ctsrbm_image_transform_corr)

In [4]:
ctsrbm_image_transform

PvtImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "PvtImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 448,
    "width": 448
  }
}

In [5]:
input = ctsrbm_dataset[0]

img_tensor = input[0]
item_id = input[1]

In [6]:
print(img_tensor.shape)

torch.Size([3, 448, 448])


In [7]:
item_id

24444

In [8]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output
    return hook

model.encoder.block[0].register_forward_hook(get_activation('encoder.block[0]'))
model.encoder.block[1].register_forward_hook(get_activation('encoder.block[1]'))
model.encoder.block[2].register_forward_hook(get_activation('encoder.block[2]'))
model.encoder.block[3].register_forward_hook(get_activation('encoder.block[3]'))
model.encoder.layer_norm.register_forward_hook(get_activation('encoder.layer_norm'))
model.encoder.register_forward_hook(get_activation('encoder'))

In [9]:
input = ctsrbm_dataset[0][0][None, :, :224, :224]
output = model(input)

HOLA

batch_size: 1
num_channels: 3
height: 224
width: 224

height: 56
width: 56

embeddings.shape: torch.Size([1, 3136, 64])

self.cls_token is None

  CALL: interpolate_pos_encoding
  embeddings.shape: torch.Size([1, 3136, 64])
  height: 56
  width: 56

  num_patches: 3136
  self.config.image_size: 224
  self.config.image_size ** 2: 50176

  embeddings.shape: torch.Size([1, 3136, 64])
  embeddings.shape: torch.Size([1, 64, 56, 56])
  interpolated_embeddings.shape: torch.Size([1, 64, 56, 56])
  interpolated_embeddings.shape: torch.Size([1, 3136, 64])

self.position_embeddings.shape: torch.Size([1, 3136, 64])
position_embeddings.shape: torch.Size([1, 3136, 64])


---

HOLA

batch_size: 1
num_channels: 64
height: 56
width: 56

height: 28
width: 28

embeddings.shape: torch.Size([1, 784, 128])

self.cls_token is None

  CALL: interpolate_pos_encoding
  embeddings.shape: torch.Size([1, 784, 128])
  height: 28
  width: 28

  num_patches: 784
  self.config.image_size: 224
  self.config.image

In [10]:
input = ctsrbm_dataset[0][0][None, :, :448, :448]
output = model(input)

HOLA

batch_size: 1
num_channels: 3
height: 448
width: 448

height: 112
width: 112

embeddings.shape: torch.Size([1, 12544, 64])

self.cls_token is None

  CALL: interpolate_pos_encoding
  embeddings.shape: torch.Size([1, 3136, 64])
  height: 112
  width: 112

  num_patches: 12544
  self.config.image_size: 224
  self.config.image_size ** 2: 50176

  embeddings.shape: torch.Size([1, 3136, 64])
  embeddings.shape: torch.Size([1, 16, 112, 112])
  interpolated_embeddings.shape: torch.Size([1, 16, 112, 112])
  interpolated_embeddings.shape: torch.Size([1, 12544, 16])

self.position_embeddings.shape: torch.Size([1, 3136, 64])
position_embeddings.shape: torch.Size([1, 12544, 16])



RuntimeError: The size of tensor a (64) must match the size of tensor b (16) at non-singleton dimension 2